In [2]:
# warnings.filterwarnings('ignore')
import pandas as pd
import psycopg2
import numpy as np
import seaborn as sns
import warnings
import random
pd.options.display.float_format = '{:.3f}'.format

conn = psycopg2.connect(
    host="tiny.db.elephantsql.com",
    database="wvhwwxaj",
    user="wvhwwxaj",
    password="nuuVmVsPvLq8QxO4UJL7erelfFgLDoEe")

cur = conn.cursor()

sql = "SELECT * FROM boardgamegeek"
cur.execute(sql)
data = cur.fetchall()

cur.close()
conn.close()

In [10]:
#save data into "df_origin" as dataframe
df_origin = pd.DataFrame(data, columns=["id","thumbnail","image","game","description","yearpublished","minplayers","maxplayers","playingtime","minplaytime",
    "maxplaytime","minage","boardgamecategory","usersrated","average","bayesaverage","rank","averageweight"])

In [ ]:
#copy for the modeling dataset
df = df_origin.copy()
#delete column"index, id, thumbnail, image, description, playingtime, usersrated, average, bayesaverage, rank, "
df = df.drop(["id", "thumbnail", "image", "description","playingtime", "usersrated", "average", "bayesaverage", "rank"], axis = 1)

In [ ]:
#build new column 'category' for machine learning
df["category"] = None
for i in range(0,len(df)):
  temp_list = df["boardgamecategory"][i].strip("[ ]").replace("'","").split(", ")
  df["category"][i] = temp_list[random.randint(0,len(temp_list)-1)]

#delete useless column
df = df.drop(["boardgamecategory"], axis = 1)

In [ ]:
#seperate data into two parts: target or nontarget
X = df.drop(columns="game")
y = df["game"]

In [ ]:
!pip install category_encoders

In [ ]:
#using pipeline transforming the dataset for the future modeling
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(
    OrdinalEncoder(handle_missing="value"), 
    SimpleImputer(), 
    RandomForestClassifier()
)

pipe.fit(X, y)

In [ ]:
X_test = {'yearpublished': [2010],
          'minplayers': [2],
          'maxplayers': [4],
          'minplaytime': [30],
          'maxplaytime': [60],
          'minage': [8],
          'averageweight' : [4.0],
          'category' : ["Card Game"]
          }
X_test = pd.DataFrame(X_test)
X_test

In [ ]:
pipe.predict(X_test)